In [1]:
import pandas as pd
from utils import RBKPreprocessor, RBKRegressor, local_metric, write_predictions, get_mean_predictions
from sklearn.model_selection import train_test_split, KFold
from catboost import CatBoostRegressor, Pool



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
RANDOM_STATE = 44

In [3]:
train_df = pd.read_csv("data/train_dataset_train.csv", index_col=0)
test_df = pd.read_csv("data/test_dataset_test.csv", index_col=0)

train_df["is_train"] = 1
test_df["is_train"] = 0

df = pd.concat([train_df, test_df])

## Натренируем препроцессор, который подготовит исходные данные к виду удобному для модели (sparse matix)

In [4]:
%%time
preprocessor = RBKPreprocessor()
preprocessor.fit(df)


Wall time: 4.07 s


RBKPreprocessor()

## Используем свой класс RBKRegressor

Который внутри содержит три модели Catboost (по одной на каждую переменную)
Натренируем 4 модели на фолдах, а потом сделаем усреднение предсказаний

In [5]:
splitter = KFold(n_splits=4, random_state=RANDOM_STATE, shuffle=True)

In [ ]:
my_models = []
for train_index, test_index in splitter.split(train_df):
    X_train = preprocessor.transform(train_df.iloc[train_index])
    y_train = train_df.iloc[train_index][["views", "depth", "full_reads_percent"]]

    X_test = preprocessor.transform(train_df.iloc[test_index])
    y_test = train_df.iloc[test_index][["views", "depth", "full_reads_percent"]]

    model = RBKRegressor(random_seed=RANDOM_STATE, n_iterations=1000)
    model.fit(X_train, y_train, X_test, y_test)
    
    my_models.append(model)


## Оценка на обучении каждой модели, и оценка при усреднении предсказаний

In [7]:
X_train = preprocessor.transform(train_df)
y_train = train_df[["views", "depth", "full_reads_percent"]]


C:\Soft\Anaconda\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
for model in my_models:
    print(local_metric(y_train, model.predict(X_train)))

(0.7882091764147028, 0.2983433326546729, 0.26849983486078255, 0.22136600889924732)
(0.8231959125907111, 0.34750425197975565, 0.2692170182329642, 0.20647464237799132)
(0.8190823253651975, 0.3577023734484537, 0.2695018482044352, 0.19187810371230876)
(0.7544934100771286, 0.31070438763547625, 0.26974827082935554, 0.17404075161229682)


In [9]:
print(local_metric(y_train, get_mean_predictions(my_models, X_train)))

(0.847544675567384, 0.36092379726769536, 0.2746464513672954, 0.21197442693239318)


## Сделаем предсказания на тестовых данных и запишем предикт

In [10]:
X_test = preprocessor.transform(test_df)

C:\Soft\Anaconda\lib\site-packages\pandas\core\arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [11]:
preds = get_mean_predictions(my_models, X_test)

In [12]:
write_predictions(preds)

## Результат на лидерборде 0.685924